In [1]:
import sys
import os 
import torch
import pandas as pd
import torchaudio.transforms as T
import torch.optim as optim
import numpy as np

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

sys.path.append(os.path.abspath(os.path.join('..','iESPnet_SRC_main','utilities')))
from Generator import SeizureDatasetLabelTime, scale_spec, permute_spec, smoothing_label
from Model import iESPnet
from TrainEval import train_model_opt, test_model, train_model, get_thr_output, get_performance_indices
from utilit import make_weights_for_balanced_classes

/home/mrobins/micromamba/envs/env_thalamus/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [10]:
meta_data_file_v1 = '/home/mrobins/Rns_Data/PITT_PI_v1/METADATA_v1/allfiles_metadata.csv'
meta_data_file_v2 = '/home/mrobins/Rns_Data/PITT_PI_v2/METADATA_v2/allfiles_metadata.csv'

df_meta1 = pd.read_csv(meta_data_file_v1)
df_meta2 = pd.read_csv(meta_data_file_v2)

df_meta1.drop(df_meta1[df_meta1['label'] == 2].index, inplace = True)
df_meta2.drop(df_meta2[df_meta2['label'] == 2].index, inplace = True)

df_meta1.drop(df_meta1[df_meta1['rns_id']=='PIT-RNS9793'].index, inplace = True)
df_meta2.drop(df_meta2[df_meta2['rns_id']=='PIT-RNS9793'].index, inplace = True)

In [11]:
# se crea el modelo

N_CLASSES       = 1
learning_rate   = 1e-3
batch_size      = 128
epochs          = 20
num_workers     = 4

hparams = {
        "n_cnn_layers" : 3,
        "n_rnn_layers" : 3,
        "rnn_dim"      : [150, 100, 50],
        "n_class"      : N_CLASSES,
        "out_ch"       : [8,8,16],
        "dropout"      : 0.3,
        "learning_rate": learning_rate,
        "batch_size"   : batch_size,
        "num_workers"  : num_workers,
        "epochs"       : epochs
        }

model = iESPnet(hparams['n_cnn_layers'],
                hparams['n_rnn_layers'],
                hparams['rnn_dim'],
                hparams['n_class'],
                hparams['out_ch'],
                hparams['dropout'],
                )

In [12]:
s=0

patients1         = df_meta1['rns_id'].unique().tolist()
save_path1        = 'SAVEPATH_v1/'
save_runs1        = save_path1 + patients1[s] + '/runs/'
save_models1      = save_path1 + patients1[s] + '/models/'
save_predictions1 = save_path1 + patients1[s] + '/results/'
save_figs1        = save_path1 + patients1[s] + '/figs/'

patients2         = df_meta2['rns_id'].unique().tolist()
save_path2        = 'SAVEPATH_v1/'
save_runs2        = save_path2 + patients2[s] + '/runs/'
save_models2      = save_path2 + patients2[s] + '/models/'
save_predictions2 = save_path2 + patients2[s] + '/results/'
save_figs2        = save_path2 + patients2[s] + '/figs/'

In [ ]:
model_path1 = save_models1 + 'model_opt.pth'
checkpoint = torch.load(model_path1)
model.load_state_dict(checkpoint['model_state_dict'])

model_path2 = save_models2 + 'model_opt.pth'
checkpoint = torch.load(model_path2)
model.load_state_dict(checkpoint['model_state_dict'])

In [14]:
patients1[0]

'PIT-RNS1603'

In [15]:
patients2[0]

'PIT-RNS1603'

In [16]:
path_results1 = save_predictions1 + patients1[s]+ 'results.npy'
path_results1

path_results2 = save_predictions2 + patients2[s]+ 'results.npy'
path_results2

results_patient_01 = np.load(path_results1, allow_pickle=True)
results_patient_01=results_patient_01.item()
results_patient_01.keys()

results_patient_02 = np.load(path_results1, allow_pickle=True)
results_patient_02=results_patient_02.item()
results_patient_02.keys()

dict_keys(['train_losses', 'train_acupr', 'prediction_te', 'prediction_tr', 'hparams', 'threshold', 'train_size'])

In [17]:
train_losses_1  = results_patient_01.get("train_losses")
train_acupr_1   = results_patient_01.get("train_acupr") 
prediction_te_1 = results_patient_01.get("prediction_te") 
prediction_tr_1 = results_patient_01.get("prediction_tr") 

train_losses_2  = results_patient_02.get("train_losses")
train_acupr_2   = results_patient_02.get("train_acupr") 
prediction_te_2 = results_patient_02.get("prediction_te") 
prediction_tr_2 = results_patient_02.get("prediction_tr") 

In [19]:
print(prediction_te_1.keys())
print(prediction_te_2.keys())

dict_keys(['accuracy', 'f1', 'precision', 'recall', 'y_true', 'y_pred', 't_true', 't_pred', 'l_true', 'l_pred', 'proba', 'MAE_time'])
dict_keys(['accuracy', 'f1', 'precision', 'recall', 'y_true', 'y_pred', 't_true', 't_pred', 'l_true', 'l_pred', 'proba', 'MAE_time'])


In [31]:
accuracy_1  = prediction_te_1.get('accuracy')
f1_1        = prediction_te_1.get('f1')
precision_1 = prediction_te_1.get('precision')
recall_1    = prediction_te_1.get('recall')
y_true_1    = prediction_te_1.get('y_true')
y_pred_1    = prediction_te_1.get('y_pred')
t_true_1    = prediction_te_1.get('t_true')
t_pred_1    = prediction_te_1.get('t_pred')
l_true_1    = prediction_te_1.get('l_true')
l_pred_1    = prediction_te_1.get('l_pred')
proba_1     = prediction_te_1.get('proba')
MAE_time_1  = prediction_te_1.get('MAE_time')

accuracy_2  = prediction_te_2.get('accuracy')
f1_2        = prediction_te_2.get('f1')
precision_2 = prediction_te_2.get('precision')
recall_2    = prediction_te_2.get('recall')
y_true_2    = prediction_te_2.get('y_true')
y_pred_2    = prediction_te_2.get('y_pred')
t_true_2    = prediction_te_2.get('t_true')
t_pred_2    = prediction_te_2.get('t_pred')
l_true_2    = prediction_te_2.get('l_true')
l_pred_2    = prediction_te_2.get('l_pred')
proba_2     = prediction_te_2.get('proba')
MAE_time_2  = prediction_te_2.get('MAE_time')

In [32]:
a=[]
a.append(float(accuracy_1))
a.append(float(f1_1))

In [38]:
a[0]

0.8248713888643484

In [ ]:
# tengo que hacer todo este proceso pero para los 31 pacientes